In [ ]:
import rasterio
import geopandas as gpd
from rasterio.crs import CRS
from rasterio.mask import mask

geoframe =  gpd.read_file('../assets/INPUT/mant_completa.shp')
crs = 'EPSG:4326'

# Defina o CRS para o GeoDataFrame
geoframe = geoframe.set_crs(crs)

# Defina o CRS desejado (EPSG 4326 - WGS 84)
crs = CRS.from_epsg(4326)

In [ ]:
geoframe['geometry'][0]

In [20]:
with rasterio.open('../assets/processed/processed/altitude_br.tif', 'r') as src:
    # Atualizar o CRS do arquivo GeoTIFF
    #src_crs = src.crs
    print(src.bounds)

BoundingBox(left=-73.9878, bottom=-33.753535500000005, right=-32.38946399999999, top=5.2699035)


In [22]:

import rasterio
import geopandas as gpd
from rasterio.crs import CRS
from rasterio.mask import mask

geoframe =  gpd.read_file('../assets/INPUT/mant_completa.shp')
crs = 'EPSG:4326'

# Defina o CRS para o GeoDataFrame
geoframe = geoframe.set_crs(crs)

# Defina o CRS desejado (EPSG 4326 - WGS 84)
crs = CRS.from_epsg(4326)
with rasterio.open('../assets/processed/processed/altitude_br.tif', 'r+') as src:
    # Atualize o CRS do arquivo GeoTIFF
    src.crs = crs
    out_image, out_transform = mask(src, geoframe.geometry, crop=True)
    # Salve as alterações nos metadados
    src.write(src.read(), 1)

ValueError: Source shape (1, 1, 4683, 4992) is inconsistent with given indexes 1

In [21]:
# Carregar o GeoDataFrame e definir o CRS
geoframe = gpd.read_file('../assets/INPUT/mant_completa.shp')
crs = 'EPSG:4326'
geoframe = geoframe.set_crs(crs)

# Abrir o raster e definir o CRS
crs_raster = CRS.from_epsg(4326)
with rasterio.open('../assets/processed/processed/altitude_br.tif', 'r') as src:
    # Atualizar o CRS do arquivo GeoTIFF
    src_crs = src.crs
    if not src_crs == crs_raster:
        raise ValueError("O CRS do raster não é o esperado (EPSG:4326)")

    # Obter os limites (BoundingBox) do raster
    raster_bounds = src.bounds

    # Verificar a interseção entre a geometria e a extensão do raster
    intersection = geoframe.geometry.intersection(raster_bounds)

    # Verificar se há alguma interseção válida
    if intersection.is_empty.all():
        raise ValueError("A geometria do GeoDataFrame não se sobrepõe ao raster")

    # Filtrar apenas geometrias que intersectam com os limites do raster
    intersecting_geometries = geoframe[intersection.not_empty]

    # Aplicar a função mask apenas se houver interseção
    out_image, out_transform = mask(src, intersecting_geometries.geometry, crop=True)
    out_meta = src.meta.copy()




ValueError: O CRS do raster não é o esperado (EPSG:4326)

In [17]:
import rasterio
import geopandas as gpd
from rasterio.crs import CRS
from rasterio.mask import mask
from shapely.geometry import box

# Carregar o GeoDataFrame e definir o CRS
geoframe = gpd.read_file('../assets/INPUT/mant_completa.shp')
crs = 'EPSG:4326'
geoframe = geoframe.set_crs(crs)

# Abrir o raster e definir o CRS
crs_raster = CRS.from_epsg(4326)
with rasterio.open('../assets/processed/processed/altitude_br.tif', 'r') as src:
    # Atualizar o CRS do arquivo GeoTIFF
    src_crs = src.crs
    if not src_crs == crs_raster:
        raise ValueError("O CRS do raster não é o esperado (EPSG:4326)")

    # Obter a geometria da BoundingBox do raster
    raster_bounds = box(*src.bounds)

    # Verificar a interseção entre a geometria e a extensão do raster
    intersection = geoframe.geometry.intersection(raster_bounds)

    # Verificar se há alguma interseção válida
    if intersection.is_empty.all():
        raise ValueError("A geometria do GeoDataFrame não se sobrepõe ao raster")

    # Filtrar apenas geometrias que intersectam com a BoundingBox do raster
    intersecting_geometries = geoframe[intersection.not_empty]

    # Aplicar a função mask apenas se houver interseção
    out_image, out_transform = mask(src, intersecting_geometries.geometry, crop=True)
    out_meta = src.meta.copy()

# Salvar as alterações nos metadados
with rasterio.open('../assets/processed/processed/altitude_br_mascara.tif', 'w', **out_meta) as dst:
    dst.write(out_image)


ValueError: O CRS do raster não é o esperado (EPSG:4326)

In [18]:
src.bounds

BoundingBox(left=-73.9878, bottom=44.2933425, right=-32.38946399999999, top=5.2699035)

In [15]:
src.bounds

BoundingBox(left=-74.0041605, bottom=43.5933705, right=-32.9974675, top=5.2699035)

In [ ]:
geoframe.geometry.intersection(raster_bounds)

In [11]:
geoframe.geometry.bounds

,minx,miny,maxx,maxy
0,-47.012839,-23.489618,-40.169585,-18.359071


In [12]:
def convert_to_decimal_degrees(degrees, minutes, seconds, direction):
    decimal_degrees = degrees + minutes / 60 + seconds / 3600
    if direction in ['S', 'W']:
        decimal_degrees *= -1
    return decimal_degrees

# Coordenadas UL (Upper Left)
ul_latitude_deg = 5
ul_latitude_min = 16
ul_latitude_sec = 15.6
ul_latitude_dir = 'N'

ul_longitude_deg = 73
ul_longitude_min = 59
ul_longitude_sec = 16.15
ul_longitude_dir = 'W'

ul_latitude_dec = convert_to_decimal_degrees(ul_latitude_deg, ul_latitude_min, ul_latitude_sec, ul_latitude_dir)
ul_longitude_dec = convert_to_decimal_degrees(ul_longitude_deg, ul_longitude_min, ul_longitude_sec, ul_longitude_dir)

# Coordenadas LR (Lower Right)
lr_latitude_deg = 33
lr_latitude_min = 45
lr_latitude_sec = 1.5
lr_latitude_dir = 'S'

lr_longitude_deg = 32
lr_longitude_min = 23
lr_longitude_sec = 16.5
lr_longitude_dir = 'W'

lr_latitude_dec = convert_to_decimal_degrees(lr_latitude_deg, lr_latitude_min, lr_latitude_sec, lr_latitude_dir)
lr_longitude_dec = convert_to_decimal_degrees(lr_longitude_deg, lr_longitude_min, lr_longitude_sec, lr_longitude_dir)

# Atribuir os valores para llx, lly, urx e ury
llx = ul_longitude_dec
lly = lr_latitude_dec
urx = lr_longitude_dec
ury = ul_latitude_dec

print("llx:", llx)
print("lly:", lly)
print("urx:", urx)
print("ury:", ury)

llx: -73.98781944444444
lly: -33.750416666666666
urx: -32.38791666666667
ury: 5.271


In [10]:
src.bounds

BoundingBox(left=-74.0041605, bottom=43.5933705, right=-32.9974675, top=5.2699035)

In [ ]:
# Salve as alterações nos metadados
with rasterio.open('../assets/processed/processed/altitude_br_mascara.tif', 'w', **out_meta) as dst:
    dst.write(out_image)

In [ ]:
crs = CRS.from_epsg(4326)

In [ ]:
out_image, out_transform = mask(src, geoframe.geometry, crop=True)

In [ ]:
src.crs

In [ ]:
geoframe.crs

In [ ]:
with rasterio.open(tiff_file) as src:
    # Máscara do arquivo .tiff usando o shapefile
    out_image, out_transform = mask(src, geoframe.geometry, crop=True)
    

In [ ]:
with rasterio.open(tiff_file) as src:
    # Máscara do arquivo .tiff usando o shapefile
    out_image, out_transform = mask(src, geoframe.geometry, crop=True)
    
    # Atualiza os metadados para refletir a nova forma e posição do recorte
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform})
    
    # Aplicar uma máscara adicional para remover os valores nodata
    out_image = np.ma.masked_where(out_image == src.nodata, out_image)

    # Salva o recorte em um novo arquivo .tiff
    with rasterio.open(output_file_name, "w", **out_meta) as dest:
        dest.write(out_image)

In [19]:
def convert_to_decimal_degrees(degrees, minutes, seconds, direction):
    decimal_degrees = degrees + minutes / 60 + seconds / 3600
    if direction in ['S', 'W']:
        decimal_degrees *= -1
    return decimal_degrees

# Coordenadas UL (Upper Left)
ul_latitude_deg = 5
ul_latitude_min = 16
ul_latitude_sec = 15.6
ul_latitude_dir = 'N'

ul_longitude_deg = 73
ul_longitude_min = 59
ul_longitude_sec = 16.15
ul_longitude_dir = 'W'

ul_latitude_dec = convert_to_decimal_degrees(ul_latitude_deg, ul_latitude_min, ul_latitude_sec, ul_latitude_dir)
ul_longitude_dec = convert_to_decimal_degrees(ul_longitude_deg, ul_longitude_min, ul_longitude_sec, ul_longitude_dir)

# Coordenadas LR (Lower Right)
lr_latitude_deg = 33
lr_latitude_min = 45
lr_latitude_sec = 1.5
lr_latitude_dir = 'S'

lr_longitude_deg = 32
lr_longitude_min = 23
lr_longitude_sec = 16.5
lr_longitude_dir = 'W'

lr_latitude_dec = convert_to_decimal_degrees(lr_latitude_deg, lr_latitude_min, lr_latitude_sec, lr_latitude_dir)
lr_longitude_dec = convert_to_decimal_degrees(lr_longitude_deg, lr_longitude_min, lr_longitude_sec, lr_longitude_dir)

# Atribuir os valores para llx, lly, urx e ury
llx = ul_longitude_dec
lly = lr_latitude_dec
urx = lr_longitude_dec
ury = ul_latitude_dec

print("llx:", llx)
print("lly:", lly)
print("urx:", urx)
print("ury:", ury)

llx: -73.98781944444444
lly: -33.750416666666666
urx: -32.38791666666667
ury: 5.271
